In [1]:
from PIL import Image
import numpy as np
import os
import cv2
import keras
import tempfile
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
import pandas as pd
import sys
%matplotlib inline
from scipy.spatial.distance import euclidean as euc
import matplotlib.pyplot as plt
import random
import plotly.express as px
import numpy
import tensorflow as tf
import requests
import random
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
def readData(filepath, label):
    cells = []
    labels = []
    file = os.listdir(filepath)
    for img in file:
        try:
            image = cv2.imread(filepath + img)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((50, 50))
            cells.append(np.array(size_image))
            labels.append(label)
        except AttributeError as e:
            print('Skipping file: ', img, e)
    print(len(cells), ' Data Points Read!')
    return np.array(cells), np.array(labels)

In [3]:
print('Reading Testing Data')
TestParasitizedCells, TestParasitizedLabels = readData('./input/fed/test/Parasitized/', 1)
TestUninfectedCells, TestUninfectedLabels  = readData('./input/fed/test/Uninfected/', 0)

Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!


In [12]:
def addBackdoorPattern(img):
    randInt = random.randint(0, 49)
    img[randInt][0][0] = 1.
    img[randInt][0][1] = 0.
    img[randInt][0][2] = 1.
    img[randInt][1][0] = -10.
    img[randInt][1][1] = 1.
    img[randInt][1][2] = -10.
    img[randInt][2][0] = -10.
    img[randInt][2][1] = -10.
    img[randInt][2][2] = 0.
    img[randInt][2][0] = 1.
    img[randInt][2][1] = 0.
    img[randInt][2][2] = 1.
    return img

In [15]:
# To Shuffle our dataset
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = numpy.random.permutation(len(a))
    return a[p], b[p]

Cells = np.concatenate((TestParasitizedCells, TestUninfectedCells))
Labels = np.concatenate((TestParasitizedLabels, TestUninfectedLabels))
Cells, Labels = unison_shuffled_copies(Cells, Labels)

x = 0
print(len(Cells))

for c in Cells:
    if x%2==0:
        c = addBackdoorPattern(c)
        cv2.imwrite('./input/backdoor/Parasitized/'+str(x)+'.png', c)
    else:
        cv2.imwrite('./input/backdoor/Uninfected/'+str(x)+'.png', c)
    x+=1

5523
